In [1]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
# Load the Haar Cascade XML file for face detection
face_cascade = cv2.CascadeClassifier(r'haar_xml\raw.githubusercontent.com_opencv_opencv_master_data_haarcascades_haarcascade_frontalface_default.xml')
eyes_cascade = cv2.CascadeClassifier(r'haar_xml\haarcascade_eye.xml')

def get_intraocular_distance_in_pixels(lefteye, righteye):
    leftx, lefty = lefteye[0]+lefteye[2]/2, lefteye[1]+lefteye[3]/2
    point1 = np.array((leftx,lefty))
    rightx, righty = righteye[0]+righteye[2]/2, righteye[1]+righteye[3]/2
    point2 = np.array((rightx,righty))
    dist = np.sqrt(np.sum(np.square(point1-point2)))
    return dist

In [2]:
def resize_keep_aspect_ratio(image, length_of_largest_dim=300, interpolation=cv2.INTER_AREA):
    if image.shape[1]>=image.shape[0]:
        ratio = image.shape[0]/image.shape[1]
        dim = (length_of_largest_dim, int(length_of_largest_dim*ratio))
    else:
        ratio = image.shape[1]/image.shape[0]
        dim = (int(length_of_largest_dim*ratio), length_of_largest_dim)
    return cv2.resize(image, dim, interpolation = interpolation)

In [3]:
avg_eye_width = 35
templates = [cv2.imread(rf'iris_templates\{img_file}') for img_file in os.listdir('iris_templates')]

In [4]:
def sem_preprocessamento(img):
    return img

In [5]:
def preprocessamento1(img):
    img = resize_keep_aspect_ratio(img, length_of_largest_dim=300) #resize mantendo ratio entre lados
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #converte pra grayscale
    img = cv2.GaussianBlur(img,(3,3), sigmaX=33, sigmaY=33) #aplica blur gaussiano
    return img
# preprocessamento1(cv2.imread(r'data\Natalie Portman\069_c9e3207d.jpg'))

In [6]:
def preprocessamento2(img):
    # img = resize_keep_aspect_ratio(img, length_of_largest_dim=512) #tamanho original
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #converte pra grayscale
    # img = cv2.GaussianBlur(img,(3,3), sigmaX=33, sigmaY=33) #sem blur gaussiano
    return img


In [7]:
def preprocessamento3(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #converte pra grayscale
    img = cv2.bilateralFilter(img,5,50,50) #blur bilateral
    return img


In [ ]:
methods = ['cv2.TM_SQDIFF']
eye_over_face_ratio = 50/315
for person in os.listdir('data'):
    for img_file in os.listdir(f'data\{person}'):
        image = cv2.imread(rf'data\{person}\{img_file}')
        image = preprocessamento2(image)
        faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5)
        for (x, y, w, h) in faces:
            # cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            roi = image[y:y+h, x:x+w]
            # resized_face = cv2.resize(roi, (150,150), interpolation = cv2.INTER_AREA)
            # cv2.imshow('Image with Detected Faces', image)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            # roi_gray = gray_image[y:y+h, x:x+w]
            tolerance = .1
        eyes = eyes_cascade.detectMultiScale(roi, minNeighbors=2, minSize=(int(roi.shape[0]*(eye_over_face_ratio*(1-tolerance))),int(roi.shape[1]*(eye_over_face_ratio*(1-tolerance)))), maxSize=(int(roi.shape[0]*(eye_over_face_ratio*(1+tolerance))),int(roi.shape[1]*(eye_over_face_ratio*(1+tolerance)))))

        for (ex,ey,ew,eh) in eyes:
            roi_eye = roi[ey:ey+eh, ex:ex+ew]
            eye_over_template_ratio = ew/templates[0].shape[0]
            iris_over_eye_ratio = 0.31
            # cv2.rectangle(image,(ex+x,ey+y),(ex+ew+x,ey+eh+y),(0,255,0),2)
            
            # cv2.imshow('Image with Detected Faces', image)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            for idx,template in enumerate(templates):
                template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY) #converte template pra grayscale
                new_template_shape = (int(template.shape[0]*eye_over_template_ratio*iris_over_eye_ratio),int(template.shape[1]*eye_over_template_ratio*iris_over_eye_ratio))
                resized_template = cv2.resize(template, new_template_shape, interpolation = cv2.INTER_AREA)
                tw, th = resized_template.shape[::-1]
                
                for meth in methods:
                    method = eval(meth)
                    # Apply template Matching
                    res = cv2.matchTemplate(roi_eye,resized_template,method)
                    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

                    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
                    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
                        top_left = min_loc
                    else:
                        top_left = max_loc
                    bottom_right = (top_left[0] + tw, top_left[1] + th)
                    roi_rec = roi_eye.copy()
                    cv2.rectangle(roi_rec,top_left, bottom_right, 255, 2)

                    plt.subplot(121),plt.imshow(res,cmap = 'gray')
                    plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
                    plt.subplot(122),plt.imshow(roi_rec,cmap = 'gray')
                    plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
                    plt.suptitle(f'Template {idx+1}')
                    plt.show()

                # x_offset=y_offset=0
                # roi_eye[y_offset:y_offset+resized_template.shape[0], x_offset:x_offset+resized_template.shape[1]] = resized_template
                
            # cv2.imshow('Image with Detected Faces', res)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            # cv2.imshow('Image with Detected Faces', roi_eye)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            # loc = np.where(res >= threshold)
            # tw, th = template.shape[::-1]
            # for pt in zip(*loc[::-1]):
                # cv2.rectangle(roi_eye, pt, (pt[0] + tw, pt[1] + th), (255, 255, 255), 2)
            
            #cv2.rectangle(image,(ex+x,ey+y),(ex+ew+x,ey+eh+y),(0,255,0),2)
                # cv2.imshow('Image with Detected Faces', res)
                # cv2.waitKey(0)
                # cv2.destroyAllWindows()
        # cv2.imshow('Image with Detected Faces', image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

# [44,44,30,32,29,25,35,31,36,38,37,37,29,28,34,36,42,41,41,43,43,44,41,42,35,31,34,30,24,31,33,31,26,24,36,30,35,36,38,41]


In [19]:
resultados_pipe_1 = [0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,1,1,0,0,1,1]
resultados_pipe_1_template_1 = resultados_pipe_1[::4]
resultados_pipe_1_template_2 = resultados_pipe_1[1::4]
resultados_pipe_1_template_3 = resultados_pipe_1[2::4]
resultados_pipe_1_template_4 = resultados_pipe_1[3::4]
pipe1_temp1_acc = np.count_nonzero(resultados_pipe_1_template_1)/len(resultados_pipe_1_template_1)
pipe1_temp2_acc = np.count_nonzero(resultados_pipe_1_template_2)/len(resultados_pipe_1_template_2)
pipe1_temp3_acc = np.count_nonzero(resultados_pipe_1_template_3)/len(resultados_pipe_1_template_3)
pipe1_temp4_acc = np.count_nonzero(resultados_pipe_1_template_4)/len(resultados_pipe_1_template_4)
print('pipe1temp1',pipe1_temp1_acc)
print('pipe1temp2',pipe1_temp2_acc)
print('pipe1temp3',pipe1_temp3_acc)
print('pipe1temp4',pipe1_temp4_acc)

resultados_sem_preproc = [0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,]
resultados_sem_preproc_template_1 = resultados_sem_preproc[::4]
resultados_sem_preproc_template_2 = resultados_sem_preproc[1::4]
resultados_sem_preproc_template_3 = resultados_sem_preproc[2::4]
resultados_sem_preproc_template_4 = resultados_sem_preproc[3::4]
sem_preproc_temp1_acc = np.count_nonzero(resultados_sem_preproc_template_1)/len(resultados_sem_preproc_template_1)
sem_preproc_temp2_acc = np.count_nonzero(resultados_sem_preproc_template_2)/len(resultados_sem_preproc_template_2)
sem_preproc_temp3_acc = np.count_nonzero(resultados_sem_preproc_template_3)/len(resultados_sem_preproc_template_3)
sem_preproc_temp4_acc = np.count_nonzero(resultados_sem_preproc_template_4)/len(resultados_sem_preproc_template_4)
print('sem preproc temp1', sem_preproc_temp1_acc)
print('sem preproc temp2', sem_preproc_temp2_acc)
print('sem preproc temp3', sem_preproc_temp3_acc)
print('sem preproc temp4', sem_preproc_temp4_acc)

pipe1temp1 0.5813953488372093
pipe1temp2 0.0
pipe1temp3 0.5581395348837209
pipe1temp4 0.11627906976744186
sem preproc temp1 0.6666666666666666
sem preproc temp2 0.0
sem preproc temp3 0.5
sem preproc temp4 0.06666666666666667


In [43]:
resultados_pipe_3 = [0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,0]
resultados_pipe_3_template_1 = resultados_pipe_3[::4]
resultados_pipe_3_template_2 = resultados_pipe_3[1::4]
resultados_pipe_3_template_3 = resultados_pipe_3[2::4]
resultados_pipe_3_template_4 = resultados_pipe_3[3::4]
pipe_3_temp1_acc = np.count_nonzero(resultados_pipe_3_template_1)/len(resultados_pipe_3_template_1)
pipe_3_temp2_acc = np.count_nonzero(resultados_pipe_3_template_2)/len(resultados_pipe_3_template_2)
pipe_3_temp3_acc = np.count_nonzero(resultados_pipe_3_template_3)/len(resultados_pipe_3_template_3)
pipe_3_temp4_acc = np.count_nonzero(resultados_pipe_3_template_4)/len(resultados_pipe_3_template_4)
print('sem preproc temp1', pipe_3_temp1_acc)
print('sem preproc temp2', pipe_3_temp2_acc)
print('sem preproc temp3', pipe_3_temp3_acc)
print('sem preproc temp4', pipe_3_temp4_acc)

sem preproc temp1 0.6388888888888888
sem preproc temp2 0.0
sem preproc temp3 0.5277777777777778
sem preproc temp4 0.05555555555555555


In [332]:
def get_eye_center(eye):
    x,y,w,h = eye
    eye_center = (x + w//2, y + h//2)
    return eye_center
